Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

batch_size = 128
def runSession(myGraph):
    num_steps = 10001#3001

    with tf.Session(graph=myGraph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [17]:
beta = 0.002
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [18]:
numHiddenNodes = 1024
nnGraph = tf.Graph()
with nnGraph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size)) #[128, 784]
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #[128, 10]
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hiddenWeights = tf.Variable(
    tf.truncated_normal([image_size * image_size, numHiddenNodes])) #[784, 1024]
  hiddenBiases = tf.Variable(tf.zeros([numHiddenNodes])) #[1024]
    
  hiddenRelu = tf.nn.relu(tf.matmul(tf_train_dataset, hiddenWeights) + hiddenBiases) # [128,784]x[784,1024]=[128, 1024]

  outputWeights = tf.Variable(
    tf.truncated_normal([numHiddenNodes, num_labels])) #[1024,10]
  outputBiases = tf.Variable(tf.zeros([num_labels])) #[10]

  # Training computation.
  outputLogits = tf.matmul(hiddenRelu, outputWeights) + outputBiases #[128,1024]x[1024,10]=[128,10]
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=outputLogits)) + beta*(tf.nn.l2_loss(hiddenWeights)
                                                                                                  +tf.nn.l2_loss(outputWeights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(outputLogits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hiddenWeights) + hiddenBiases)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_hidden, outputWeights) + outputBiases)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hiddenWeights) + hiddenBiases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, outputWeights) + outputBiases)

In [22]:
runSession(graph)
runSession(nnGraph)

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
runSession(nnGraph)

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [7]:
numHiddenNodes = 1024
beta = 0.002
nnGraph = tf.Graph()
with nnGraph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size)) #[128, 784]
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #[128, 10]
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hiddenWeights = tf.Variable(
    tf.truncated_normal([image_size * image_size, numHiddenNodes])) #[784, 1024]
  hiddenBiases = tf.Variable(tf.zeros([numHiddenNodes])) #[1024]
    
  hiddenRelu = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, hiddenWeights) + hiddenBiases), keep_prob=0.5) # [128,784]x[784,1024]=[128, 1024]

  outputWeights = tf.Variable(
    tf.truncated_normal([numHiddenNodes, num_labels])) #[1024,10]
  outputBiases = tf.Variable(tf.zeros([num_labels])) #[10]

  # Training computation.
  outputLogits = tf.matmul(hiddenRelu, outputWeights) + outputBiases #[128,1024]x[1024,10]=[128,10]
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=outputLogits)) #+ beta*(tf.nn.l2_loss(hiddenWeights)
                                                                                                  #+tf.nn.l2_loss(outputWeights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, hiddenWeights) + hiddenBiases)
  train_prediction = tf.nn.softmax(
    tf.matmul(train_hidden, outputWeights) + outputBiases)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hiddenWeights) + hiddenBiases)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_hidden, outputWeights) + outputBiases)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hiddenWeights) + hiddenBiases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, outputWeights) + outputBiases)

In [8]:
runSession(nnGraph)

Initialized
Minibatch loss at step 0: 437.984863
Minibatch accuracy: 14.8%
Validation accuracy: 31.8%
Minibatch loss at step 500: 29.761795
Minibatch accuracy: 77.3%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 25.692440
Minibatch accuracy: 71.1%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 10.936906
Minibatch accuracy: 82.0%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 10.848547
Minibatch accuracy: 86.7%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 11.186253
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 4.625934
Minibatch accuracy: 78.1%
Validation accuracy: 79.8%
Minibatch loss at step 3500: 2.740664
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 4000: 3.802878
Minibatch accuracy: 82.0%
Validation accuracy: 79.9%
Minibatch loss at step 4500: 6.610671
Minibatch accuracy: 76.6%
Validation accuracy: 81.3%
Minibatch loss at step 5000: 6.493934
Minibatch accuracy: 81.2%
Validation 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [169]:
layer1Nodes = 1024
layer2Nodes = 1024
layer3Nodes = 256
layer4Nodes = 64
beta = 0.002
initial_learning_rate_value = 0.5
nnGraph = tf.Graph()

def feedForward(inputData, paramList=[]):
    activation = inputData
    for weights, bias in paramList:
        activation = tf.nn.relu(tf.matmul(activation, weights) + bias)
    return activation
    
def computeL2Loss(weightList=[]):
    lossSum = 0
    for weights in weightList:
        lossSum += tf.nn.l2_loss(weights)
        
    return beta*lossSum
    
def fullyConnectedLayer(inputData, dataSize, numNodes, keepProb=0.5):
    hiddenWeights = tf.Variable(
    tf.truncated_normal([dataSize, numNodes], stddev=math.sqrt(2.0/dataSize))) #[784, 1024]
    hiddenBiases = tf.Variable(tf.zeros([numNodes])) #[1024]
    
    return tf.nn.dropout(tf.nn.relu(tf.matmul(inputData, hiddenWeights) + hiddenBiases), 
                         keep_prob=keepProb), hiddenWeights, hiddenBiases
    #return tf.nn.relu(tf.matmul(inputData, hiddenWeights) + hiddenBiases), hiddenWeights, hiddenBiases  

with nnGraph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size)) #[128, 784]
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #[128, 10]
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1Relu, layer1Weights, layer1Bias = fullyConnectedLayer(tf_train_dataset, image_size*image_size, layer1Nodes, 0.5)
  layer2Relu, layer2Weights, layer2Bias = fullyConnectedLayer(layer1Relu, layer1Nodes, layer2Nodes, 0.75)
  layer3Relu, layer3Weights, layer3Bias = fullyConnectedLayer(layer2Relu, layer2Nodes, layer3Nodes, 0.8)
  layer4Relu, layer4Weights, layer4Bias = fullyConnectedLayer(layer3Relu, layer3Nodes, layer4Nodes, 0.9)

  outputWeights = tf.Variable(
    tf.truncated_normal([layer4Nodes, num_labels], stddev=math.sqrt(2.0/layer4Nodes))) #[1024,10]
  outputBiases = tf.Variable(tf.zeros([num_labels])) #[10]

  # Training computation.
  outputLogits = tf.matmul(layer4Relu, outputWeights) + outputBiases #[128,1024]x[1024,10]=[128,10]
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, 
                                            logits=outputLogits)) + computeL2Loss([layer1Weights, 
                                                                                   layer2Weights, layer3Weights, layer4Weights, outputWeights])
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(initial_learning_rate_value, global_step, 1, 0.9999)

  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  paramList = [(layer1Weights, layer1Bias), (layer2Weights, layer2Bias), (layer3Weights, layer3Bias), (layer4Weights, layer4Bias)]
  #paramList = [(layer2Weights, layer2Bias)]
  train_hidden = feedForward(tf_train_dataset, paramList)
  train_prediction = tf.nn.softmax(
    tf.matmul(train_hidden, outputWeights) + outputBiases)
  valid_hidden = feedForward(tf_valid_dataset, paramList)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_hidden, outputWeights) + outputBiases)
  test_hidden = feedForward(tf_test_dataset, paramList)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, outputWeights) + outputBiases)

In [170]:
runSession(nnGraph)

Initialized
Minibatch loss at step 0: 6.332182
Minibatch accuracy: 7.8%
Validation accuracy: 13.8%
Minibatch loss at step 500: 2.177239
Minibatch accuracy: 82.8%
Validation accuracy: 84.5%
Minibatch loss at step 1000: 1.408041
Minibatch accuracy: 77.3%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 0.906124
Minibatch accuracy: 85.9%
Validation accuracy: 85.3%
Minibatch loss at step 2000: 0.678983
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 2500: 0.819011
Minibatch accuracy: 84.4%
Validation accuracy: 86.4%
Minibatch loss at step 3000: 0.868221
Minibatch accuracy: 81.2%
Validation accuracy: 86.3%
Minibatch loss at step 3500: 0.560578
Minibatch accuracy: 89.1%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.783317
Minibatch accuracy: 82.8%
Validation accuracy: 87.4%
Minibatch loss at step 4500: 0.706017
Minibatch accuracy: 82.8%
Validation accuracy: 87.5%
Minibatch loss at step 5000: 0.521253
Minibatch accuracy: 90.6%
Validation accuracy